# Find features with missing attachments

This example shows how to find all features in a feature layer that do not have an attached image. Features that are missing an image are added to a view layer so that crews can go out and re-inspect them.

#### Import the ArcGIS API for Python

In [1]:
import arcgis

#### Connect to ArcGIS Online and get the damage assessment layer and its view layer

In [2]:
gis = arcgis.gis.GIS("https://arcgis.com", "aaron_nitro")
damage_assessments_layer = gis.content.get('d46499afaa9142c0bd261478d329c027').layers[0]
damage_assessments_view_layer = gis.content.get('d85d43a523f940079982a076c944405f').layers[2]

Enter password: ········


#### Get all damage assessments performed in the last 10 days

In [3]:
where = f"{damage_assessments_layer.properties.editFieldsInfo.creationDateField} >= CURRENT_TIMESTAMP - INTERVAL '10' DAY"
damage_assessments = damage_assessments_layer.query(where=where)

#### Create a set of all the globalids

In [4]:
global_ids = {da.attributes[damage_assessments_layer.properties.globalIdField] for da in damage_assessments}

#### Query all of the attachments collected within the last 10 days and create a set of their parent feature globalids

In [5]:
attachments = damage_assessments_layer.attachments.search(where=where, attachment_types=['image/jpeg', 'image/jpg', 'image/png'])
parent_global_ids = {x['PARENTGLOBALID'] for x in attachments}

#### Find the globalids which do not overlap which indicates it's missing an attachment

In [6]:
global_ids_without_an_image = global_ids.difference(parent_global_ids)

#### Create a SQL where clause using the globalids

In [8]:
where = f"""{damage_assessments_layer.properties.globalIdField} in ('{",".join(global_ids_without_an_image)}')"""
where

"GlobalID in ('c4c03f20-fb04-47b3-88f6-80b6ad1d339b')"

#### Update the view definition query of the view layer to only show features that are missing attachments

In [9]:
damage_assessments_view_layer.manager.update_definition({
    "viewDefinitionQuery": where
})

{'success': True}